In [1]:
import pandas as pd
import urllib
import os
import string
import re
import numpy as np

In [2]:
df=pd.read_csv("Data/df_final.csv", sep=";", index_col=0)
df.head()

,DATE ARRET,CDGI,QUARTIER,TYPE DE VOIE,PREFIXE de VOIE,NOM de VOIE,Libellé VOIE,CVOIE,HISTORIQUE,Personne,...,Voies,Eponymes,Arrondissements,Origine rue,Date,Origine_rue,Date_personnalité,Femme,Homme,Sexe
0,1864-02-03T00:53:28+00:53,4166,Roquette.,rue,NaN,gerbier,rue gerbier,4112,"pierre jean baptiste gerbier (1725-1788), avoc...",Pierre Jean Baptiste Gerbier,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,M
1,NaN,4994,Saint-Ambroise.,cité,NaN,joly,cité joly,4886,"m. joly, propriétaire.",M. Joly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,M
2,NaN,5441,Saint-Ambroise.,rue,NaN,léchevin,rue léchevin,5376,"m. léchevin, propriétaire.\nhistorique. ~ avan...",M. Léchevin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,M
3,NaN,8208,Folie-Méricourt. Saint-Ambroise. Roquette.,boulevard,NaN,richard lenoir,boulevard richard lenoir,8197,"françois richard (1765-1839), manufacturier d'...",François Richard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,M
4,1885-10-11T00:53:28+00:53,9057,Saint-Ambroise.,rue,NaN,spinoza,rue spinoza,8671,"baruch spinoza (1632-1677), philosophe holland...",Baruch Spinoza,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,M


# 1. Arrondissements

## 1.1. Clean

On cherche à compter les rues féminines et masculines par arrondissement. Or, certaines rues sont à la frontière entre plusieurs arrondissements. On va considérer que ces rues sont alors dans les 2 arrondissements - elles seront alors comptées plusieurs fois au total, mais une fois dans chaque arrondissement.

In [3]:
df_arrondissements = pd.concat([pd.Series(row['Sexe'], row['ARRONDISSEMENT'].split(','))              
                    for _, row in df.iterrows()]).reset_index().rename(columns={'index' : 'Arrondissement', 0:'Sexe'})

In [4]:
df_arrondissements['Arrondissement'] = df_arrondissements['Arrondissement'].apply(lambda x: x.upper())
removed_expressions = [' ', 'E', 'R']
for expression in removed_expressions:
    df_arrondissements['Arrondissement'] = df_arrondissements['Arrondissement'].apply(lambda x: x.replace(expression, ''))
df_arrondissements = df_arrondissements.loc[~ df_arrondissements['Arrondissement'].str.startswith("LIMIT")]

In [5]:
def from_roman(num):
    roman_numerals = {'I':1, 'V':5, 'X':10, 'L':50, 'C':100, 'D':500, 'M':1000}
    try:
        result = 0
        for i,c in enumerate(num):
            if (i+1) == len(num) or roman_numerals[c] >= roman_numerals[num[i+1]]:
                result += roman_numerals[c]
            else:
                result -= roman_numerals[c]
    except:
        result = int(num)
    return result
df_arrondissements['Arrondissement_num'] = df_arrondissements['Arrondissement'].apply(lambda x: from_roman(x))
df_arrondissements=df_arrondissements.loc[df_arrondissements['Arrondissement_num']!=0]

## 1.2. Nombre de rues / sexe par arrondissement

In [6]:
df_arrondissements = pd.crosstab(df_arrondissements['Arrondissement_num'], df_arrondissements['Sexe']).reset_index()
df_arrondissements.columns.name=None

In [7]:
df_arrondissements['Total'] = df_arrondissements[['M', 'F']].sum(axis=1)

In [8]:
df_arrondissements['F_percent'] = df_arrondissements['F']/df_arrondissements['Total']*100

In [9]:
df_arrondissements[['Arrondissement_num', 'F_percent']].to_csv('data/df_arrondissements.csv')

# 2. Quartiers de Paris

## 2.1. Clean

In [10]:
df_quartier = df
# On enlève le point si le quartier finit par un point
pattern_finish_point = re.compile('\.$')
df_quartier['Quartier'] = df_quartier['QUARTIER'].apply(lambda x: pattern_finish_point.sub('', x))

In [11]:
for punctuation in ['.', ',', ';'] :
    df_quartier = pd.concat([pd.Series(row['Sexe'], row['Quartier'].split(punctuation))              
                    for _, row in df_quartier.iterrows()]).reset_index().rename(columns={'index' : 'Quartier', 0:'Sexe'})

In [12]:
# "Sur le territoire de..." à enlever, ce n'est pas un district
df_quartier = df_quartier[~df_quartier['Quartier'].str.contains('Sur le|Situé sur')]

# Enlever espace au début ; mettre en majuscule ; ajouter '-' entre les mots plutôt que espaces
df_quartier['Quartier'] = df_quartier['Quartier'].apply(lambda x: x.strip().title().replace(' ', '-').replace('--', '-'))

In [13]:
# Correction à la main des erreurs qu'il reste
corrections = {'Folies-Méricourt' : 'Folie-Méricourt',
              "Goutte-D\x92Or" : "Goutte-D'Or", 
              "Hôpit" : "Hôpital-Saint-Louis", 
              "La-Villette" : "Villette", 
              "Plaine-De-Monceau" : "Plaine-De-Monceaux", 
              "Saint-Germain-Des-Pré" : "Saint-Germain-Des-Prés", 
              "Épinettes" : "Epinettes", 
              "Les-Halles" : 'Halles',
              "Chapelle" : "La Chapelle", 
              "Saint-Germain-L'Auxerrois" : "St-Germain-L'Auxerrois", 
              'Lambert' : 'Saint-Lambert'}

for erreur in corrections.keys():
    df_quartier.loc[df_quartier['Quartier']==erreur, 'Quartier'] = corrections.get(erreur)

## 2.2. Nombre de rues/sexe par quartier

In [14]:
df_quartier = pd.crosstab(df_quartier['Quartier'], df_quartier['Sexe']).reset_index()
df_quartier.columns.name=None

In [15]:
df_quartier['Total'] = df_quartier[['M', 'F']].sum(axis=1)

In [16]:
df_quartier['F_percent'] = df_quartier['F']/df_quartier['Total']*100

In [17]:
df_quartier[['Quartier', 'F_percent']].to_csv('data/df_quartier.csv')

# 3. Professions

## 3.1. Wordsclouds femmes vs hommes

In [18]:
import stylecloud

In [19]:
for element in ['[', ']', '"', "'"]:
    df.loc[~df['JOBS_list'].isna(), 'JOBS_list'] = df.loc[~df['JOBS_list'].isna(), 'JOBS_list'].apply(lambda x: x.replace(element, ""))
df.loc[~df['JOBS_list'].isna(), 'JOBS_list'] = df.loc[~df['JOBS_list'].isna(), 'JOBS_list'].apply(lambda x: x.split(','))

In [20]:
jobs_list_w = [item for sublist in df.loc[(~df['JOBS_list'].isna()) & (df['Sexe']=='F'), 'JOBS_list'].tolist() for item in sublist]
jobs_list_w = [x.strip(' ') for x in jobs_list_w]

jobs_list_m = [item for sublist in df.loc[(~df['JOBS_list'].isna()) & (df['Sexe']=='M'), 'JOBS_list'].tolist() for item in sublist]
jobs_list_m = [x.strip(' ') for x in jobs_list_m]

In [21]:
text_f=' '.join(jobs_list_w)
text_m=' '.join(jobs_list_m)
file = open("Wordclouds/text_f.txt",'w',  encoding="utf-8") 
file.write(text_f) 
file.close()
file = open("Wordclouds/text_m.txt",'w',  encoding="utf-8") 
file.write(text_m) 
file.close()

In [22]:
stylecloud.gen_stylecloud(file_path="Wordclouds/text_f.txt", 
                        icon_name="fas fa-female", 
                        palette='cartocolors.sequential.Magenta_7')

In [23]:
stylecloud.gen_stylecloud(file_path="Wordclouds/text_m.txt", 
                        icon_name="fas fa-male", 
                        palette='cmocean.sequential.Ice_15_r')

## 3.2. Cleaner les données de jobs?

In [24]:
jobs_list = [item for sublist in df.loc[~df['JOBS_list'].isna(), 'JOBS_list'].tolist() for item in sublist]
jobs_list = set([x.strip(' ') for x in jobs_list])

In [25]:
# Résistants : résistant, otage, résistance, 
# syndicaliste : syndicaliste, militant
# féministe : féministe,
# Titre de nobelesse : comte, marquis, duc, vicomte, seigneur, baron


# homme politique : homme politique, femme politique, maire, conseiller municipal, conseiller, conseillère, préfet, homme d'etat, homme d'état, 
                #homme detat, homme détat, ministre, sénateur, empereur, roi, président, prince, ambassadeur, diplomate, 
                #député, magistrat
# Justice : avocat, jurisconsulte, greffier, juriste, notaire
# Proprétaire : propriétaire, proprietaire, entrepreneur, concessionnaire, gérant
# Artisan/manufacture : manufacturier, industriel, apothicaire, ciseleur, fondeur, graveur, mécanicien, cuisinier, gastronome,
        #horticulteur, imprimeur

# Intellectuel : écrivain, écrivaine, romancier, lithographe,  littérateur, linguiste, anthropologue, érudit, 
            #philosophe, lexicographe, poète, sociologue, sociologiste, économiste, critique, auteur, auteure, dramaturge, 
            #essayiste, homme de lettres, auteur, savant, historien, journaliste, homme de lettres, femme de lettres, calligraphe,
            #chroniqueur, fabuliste, helléniste, instituteur, institutrice, poétesse
            
# Artiste : peintre, graveur, compositeur, compositrice, interprète, dessinateur, dessinatrice, chansonnier, chansonnière, 
            #tragédien, statuaire, sculpteur, danseur, danseuse, décorateur, femme peintre, illustrateur
            # architecte, acteur, actrice, comédien, comédienne, musicien, clarinettiste, guitariste,
            #saxophoniste, violoniste, chef d'orchestre, cantatrice, chanteur, chanteuse, caricaturiste, chorégraphe, cinéaste,
            #compositrice, facteur, humoriste, jardinier
        
# Aviateur : aviateur, aviatrice, pilote, corsaire, marin, navigateur
# Sportif: champion, gymnastique
# Militaire : général, colonel, militaire, amiral, maréchal, officier, capitaine, sergent, lieutenant, gardien de la paix, 
                # homme de guerre
            
# Scientifique : mathématicien, physicien, chimiste, minéarologiste, botaniste, explorateur, ingénieur, inventeur, 
            #aéronaute, naturaliste, voyageur, géologue, agronome, alchimiste, bactériologiste, savant, géographe,
            # archéologue,démographe, ethnographe, ethnologue, géomètre, informaticienne, microbiologiste
# Médecin : docteur, médecin, chirurgien, neurologue, cancérologue, anatomiste, infirmière
# Entourage royal : argentier, chancelier
# Religion : saint, prêtre, pretre, archevêque, archeveque, cardinal, rabbin, jésuite, curé, chef protestant, aumônier,
            #bénédictin, dominicain


In [26]:
#df.loc[(~df['HISTORIQUE'].isna()) & df['HISTORIQUE'].str.contains('intendant'), 'HISTORIQUE'].tolist()

# 4. Dates arrêtés

Y a -t-il une évolution dans le temps du choix des noms donnés aux rues? On regarde les arrêtés depuis 1980.

In [27]:
df.loc[~df['DATE ARRET'].isna(), 'year_arret'] = df.loc[~df['DATE ARRET'].isna(), 'DATE ARRET'].apply(lambda x: int(x[:4]))

In [28]:
df_dates_arret = df.loc[(df['bool_personne']==1) & (df['year_arret']>=1980), ['bool_personne', 'year_arret', 'Sexe']]
pd.crosstab(df_dates_arret['year_arret'], df_dates_arret['Sexe']).to_csv('Data/Date_arret.csv')

# 5. Dates

In [29]:
df.loc[(~df['HISTORIQUE'].isna()) & (df['HISTORIQUE'].str.contains('\(')), 
       'Parenthèses'] = df.loc[(~df['HISTORIQUE'].isna()) & (df['HISTORIQUE'].str.contains('\(')), 
                               'HISTORIQUE'].apply(lambda x: re.search(r'\((.*?)\)',x).group(1))
df.loc[~df['Parenthèses'].isna(), 'Date_fin'] = df.loc[~df['Parenthèses'].isna(), 'Parenthèses'].apply(lambda x: re.search(r'\d{4}(?!\d)', x))

In [30]:
df['Dates_parenthèses'] = df['Parenthèses'].astype(str).apply(lambda x: re.findall('\d+', x))

In [31]:
df.loc[(~df['HISTORIQUE'].isna()), 'Dates_historique'] =  df.loc[(~df['HISTORIQUE'].isna()), 'HISTORIQUE']\
                                                                            .apply(lambda x: re.findall(r'\d+', x))

In [32]:
df.loc[~df['Dates_historique'].isna(), 'Dates_historique'] = df.loc[~df['Dates_historique'].isna(), 'Dates_historique'].apply(lambda x: [i for i in x if len(i)==4])


In [33]:
df.loc[(df['bool_personne']==1) & (~df['Dates_historique'].isna()),'Dates_historique'] = df.loc[(df['bool_personne']==1) & (~df['Dates_historique'].isna()),'Dates_historique'].apply(lambda x: x[:2])

In [34]:
df.loc[(df['bool_personne']==1) & (~df['Dates_historique'].isna()),'nb_dates_historique']=df.loc[(df['bool_personne']==1) & (~df['Dates_historique'].isna()),'Dates_historique'].apply(lambda x: len(x))

In [36]:
df['naissance'] = df.loc[(df['bool_personne']==1) & (df['nb_dates_historique']==2),'Dates_historique'].apply(lambda x: x[0])